<a href="https://colab.research.google.com/github/gandesirijugalkishore/DATA-MIGRATION/blob/main/DATA_FROM_TO_ELASTIC_SEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DateType, StringType
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql import functions as sf
import time
import yaml
import pickle
from datetime import datetime, date, timedelta
from elasticsearch import Elasticsearch
import pandas as pd


def createSparkSession():
    spark_session = SparkSession.builder.appName("app").getOrCreate()
    return spark_session


def readFromElasticSearch(spark,index):
    df= spark.read\
    .format("org.elasticsearch.spark.sql")\
    .option("es.nodes","..........")\
    .load(index)
    return df.limit(100)

def writeToElasticSearch(df,es_config):
    es_host = f"http://{es_config['host']}:{es_config['port']}"
    es_user = es_config['username']
    es_pass = es_config['password']
    index_name = es_config['index_name']
    print(f'Index name : {index_name}')
    df.write.format("org.elasticsearch.spark.sql").option("es.nodes", es_host)\
                                                    .option("es.net.http.auth.user",es_user) \
                                                    .option("es.net.http.auth.pass",es_pass) \
                                                    .option("es.batch.size.entries", 60000) \
                                                    .mode("overwrite").save(index_name)
    response_data = {'response_Code':'200','response':'Inserted Sucessfully'}
    print(response_data)

In [ ]:
if __name__ == '__main__':
    # Configuration
    config_file = 'config.yml'
    with open(config_file) as f:
        config = yaml.safe_load(f)
    oracle_config = config['CONFIG+FILE']
    es_config = config['CONFIG+FILE']
    es_config['index_name'] = 'YOUR_INDEX_NAME'

    spark = createSparkSession()
    df = readFromElasticSearch(spark,es_config['index_name'])
    df.take(1)
#     writeToElasticSearch(df,es_config)